<a href="https://colab.research.google.com/github/Samvirtha/deep-learning2-section1/blob/main/Copy_of_notebook22f397e761.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'medium-articles:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2123951%2F3531703%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240908%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240908T094733Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D6549a6dea7fc63bd4ec7495076974398880554c5c7235b08ff6ce5dfea80c28f2774fa5b1a75066b4647afa5b6cb34247aa62ea20d0db67a9b3d21318a58734679e15d3a6d4ad596d857c2ed5778dab6abf89d3cae7768496969e8d8ed1eae9e04a1a5d48fcbd36599f1768a5747d34371f401cf041bdd4f4715644ba239ecb8122e72ffd91c818b9ba591d79871ba7a993a3a3f8fcab70df8deebcf6381a4a8a3f4d0d5f14b3bec33b36e616c3199bcf10c1a61939acf75e539dd5aaf808ec78f284b1b57b84c8a7ef0782b77287d2f07d4a1be8739cb3c1bea0928716364b0827adaf9b788f5757b6e3d888787d86c213f01812d180f511763414b307c2b6e'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 386824829 bytes downloaded
Downloaded and uncompressed: medium-articles
Data source import complete.


In [8]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/medium-articles/medium_articles.csv


In [9]:
import numpy as np

import pandas as pd

import nltk

import re

from nltk.tokenize import sent_tokenize

from nltk.corpus import stopwords

from gensim.models import Word2Vec

from scipy import spatial

import networkx as nx

import csv

In [10]:
df=pd.read_csv('/kaggle/input/medium-articles/medium_articles.csv')

df.head()


,title,text,url,authors,timestamp,tags
0,Mental Note Vol. 24,Photo by Josh Riemer on Unsplash\n\nMerry Chri...,https://medium.com/invisible-illness/mental-no...,['Ryan Fan'],2020-12-26 03:38:10.479000+00:00,"['Mental Health', 'Health', 'Psychology', 'Sci..."
1,Your Brain On Coronavirus,Your Brain On Coronavirus\n\nA guide to the cu...,https://medium.com/age-of-awareness/how-the-pa...,['Simon Spichak'],2020-09-23 22:10:17.126000+00:00,"['Mental Health', 'Coronavirus', 'Science', 'P..."
2,Mind Your Nose,Mind Your Nose\n\nHow smell training can chang...,https://medium.com/neodotlife/mind-your-nose-f...,[],2020-10-10 20:17:37.132000+00:00,"['Biotechnology', 'Neuroscience', 'Brain', 'We..."
3,The 4 Purposes of Dreams,Passionate about the synergy between science a...,https://medium.com/science-for-real/the-4-purp...,['Eshan Samaranayake'],2020-12-21 16:05:19.524000+00:00,"['Health', 'Neuroscience', 'Mental Health', 'P..."
4,Surviving a Rod Through the Head,"You’ve heard of him, haven’t you? Phineas Gage...",https://medium.com/live-your-life-on-purpose/s...,['Rishav Sinha'],2020-02-26 00:01:01.576000+00:00,"['Brain', 'Health', 'Development', 'Psychology..."


In [11]:
df=df[['title','text']]
df.shape

(192368, 2)

In [12]:
df=df[['title','text']]
df.shape

(192368, 2)

In [13]:
df['text']=df['text'].str.replace('\n','<|n|>')


In [14]:
sample_blog=df['text'][10]
print(sample_blog)

Photo credit: Leo Leung<|n|><|n|>People want to know why you do what you do.<|n|><|n|>If you’re an entrepreneur, your investors, customers, and employees want to know why you founded your company. Apply for a job, and recruiters ask about your career path. If you’re a freelancer, everyone is curious about how you got started.<|n|><|n|>Why do we crave origin stories? Because they reveal character. Superman is a good guy because his planet exploded, his parents sent him to Earth, and the Kents raised him to fight evil. Steve Jobs became a trusted CEO, in part, by bouncing back from failure. My dad thrived as a home builder, but only after terminating a partnership with a prominent colleague whose practices he found unethical.<|n|><|n|>In other words:<|n|><|n|>Your origin story helps people decide whether or not to trust you.<|n|><|n|>How to Craft a Powerful Origin Story: The 5 Essential Elements<|n|><|n|>In strategic messaging and positioning engagements with CEOs, as well as business st

In [15]:
!pip install nltk


In [16]:
import nltk # imports the nltk library
import re
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from gensim.models import Word2Vec
import numpy as np

nltk.download('punkt') # downloads the punkt sentence tokenizer models
nltk.download('stopwords') # downloads the stopwords corpus

sentences=sent_tokenize(sample_blog)
sentences_clean=[re.sub(r'[^\w\s]','',sentence.lower()) for sentence in sentences]
stop_words = stopwords.words('english')
sentence_tokens=[[words for words in sentence.split(' ') if words not in stop_words]
for sentence in sentences_clean]
w2v=Word2Vec(sentence_tokens,vector_size=1,min_count=1,epochs=1000)
sentence_embeddings=[[w2v.wv.get_vector(word)[0] for word in words] for words in sentence_tokens]
max_len=max([len(tokens) for tokens in sentence_tokens])
sentence_embeddings=[np.pad(embedding,(0,max_len-len(embedding)),'constant') for embedding in sentence_embeddings]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [17]:
simiarity_matrix = np.zeros([len(sentence_tokens), len(sentence_tokens)]) # misspelled variable name
for i,row_embedding in enumerate(sentence_embeddings):
    for j,column_embedding in enumerate(sentence_embeddings):
        simiarity_matrix[i][j]=1-spatial.distance.cosine(row_embedding,column_embedding)
nx_graph = nx.from_numpy_array(simiarity_matrix) # use the correctly spelled variable name
scores=nx.pagerank(nx_graph,max_iter=600)
top_sentence={sentence:scores[index] for index,sentence in enumerate(sentences)}
sentNeeded= round(0.25*len(sentences))-1
top=dict(sorted(top_sentence.items(), key=lambda x: x[1], reverse=True)[:sentNeeded])
summary=""
for sent in sentences:
  if sent in top.keys():
    summary+= sent
print(summary)

Apply for a job, and recruiters ask about your career path.If you’re a freelancer, everyone is curious about how you got started.<|n|><|n|>Why do we crave origin stories?Steve Jobs became a trusted CEO, in part, by bouncing back from failure.Very frequently, another person illuminates a new way forward.<|n|><|n|>Can you pinpoint the event that set you on your current path?The sign says, “For anyone who wants to tell a compelling story,” and it’s pointing to a book about screenwriting.Basically, describe what happened next:<|n|><|n|>That night, I read the book from cover to cover.It’s written in the language of feature films (three act structure, etc.I rewrite my pitch and send it to a new batch of investors.<|n|><|n|>4.“Until finally…”<|n|><|n|>What was the turning point — the moment when you really committed to the new path?Pay particular attention to where they lean in.<|n|><|n|>Also, ask others to tell you their origin stories.


In [18]:
simiarity_matrix = np.zeros([len(sentence_tokens), len(sentence_tokens)]) # misspelled variable name
for i,row_embedding in enumerate(sentence_embeddings):
    for j,column_embedding in enumerate(sentence_embeddings):
        # Calculate the mean of the word embeddings for each sentence
        row_mean = np.mean(row_embedding, axis=0)
        column_mean = np.mean(column_embedding, axis=0)
        # Check if row_mean and column_mean are scalars
        if row_mean.ndim == 0:
            row_mean = np.expand_dims(row_mean, axis=0)
        if column_mean.ndim == 0:
            column_mean = np.expand_dims(column_mean, axis=0)
        simiarity_matrix[i][j]=1-spatial.distance.cosine(row_mean,column_mean)

# Print the similarity matrix
print("Similarity Matrix:")
print(simiarity_matrix)

nx_graph = nx.from_numpy_array(simiarity_matrix) # use the correctly spelled variable name
scores=nx.pagerank(nx_graph,max_iter=600)

# Print the PageRank scores
print("\nPageRank Scores:")
for index, sentence in enumerate(sentences):
    print(f"{sentence}: {scores[index]}")

top_sentence={sentence:scores[index] for index,sentence in enumerate(sentences)}
sentNeeded= round(0.25*len(sentences))-1
top=dict(sorted(top_sentence.items(), key=lambda x: x[1], reverse=True)[:sentNeeded])
summary=""
for sent in sentences:
  if sent in top.keys():
    summary+= sent

# Print the summary
print("\nSummary:")
print(summary)

Similarity Matrix:
[[1.         0.99999999 1.         ... 1.         0.99999999 1.        ]
 [0.99999999 1.         1.         ... 0.99999998 0.9999999  0.99999994]
 [1.         1.         1.         ... 1.         1.         0.99999998]
 ...
 [1.         0.99999998 1.         ... 1.         0.99999996 1.        ]
 [0.99999999 0.9999999  1.         ... 0.99999996 0.99999998 0.99999994]
 [1.         0.99999994 0.99999998 ... 1.         0.99999994 0.99999999]]

PageRank Scores:
Photo credit: Leo Leung<|n|><|n|>People want to know why you do what you do.<|n|><|n|>If you’re an entrepreneur, your investors, customers, and employees want to know why you founded your company.: 0.02364718620808374
Apply for a job, and recruiters ask about your career path.: 0.023647185945244047
If you’re a freelancer, everyone is curious about how you got started.<|n|><|n|>Why do we crave origin stories?: 0.02364718633224591
Because they reveal character.: 0.023647186211972256
Superman is a good guy because hi

In [19]:
import re

from nltk.tokenize import sent_tokenize

from nltk.corpus import stopwords

from gensim.models import Word2Vec

from scipy import spatial
!pip install --upgrade networkx


import networkx as nx
import csv

In [20]:
count=0
def generateSummary(blog):
  global count
  count+=1
  print("Summarising blog", count)
  try:
    sentences=sent_tokenize(blog)
    sentences_clean=[re.sub(r'[^\w\s]','',sentence.lower()) for sentence in sentences]
    stop_words = stopwords.words('english')
    sentence_tokens=[[words for words in sentence.split(' ') if words not in stop_words] for sentence in sentences_clean]
    w2v=Word2Vec(sentence_tokens,vector_size=1,min_count=1,epochs=1000) # Removed the extra indent from this line.
    sentence_embeddings=[[w2v.wv.get_vector(word)[0] for word in words] for words in sentence_tokens]
    max_len=max([len(tokens) for tokens in sentence_tokens])
    sentence_embeddings=[np.pad(embedding,(0,max_len-len(embedding)),'constant') for embedding in sentence_embeddings]
    simiarity_matrix = np.zeros([len(sentence_tokens), len(sentence_tokens)])
    for i,row_embedding in enumerate(sentence_embeddings):
      for j,column_embedding in enumerate(sentence_embeddings):
        simiarity_matrix[i][j]=1-spatial.distance.cosine(row_embedding,columwn_embedding)
        nx_graph = nx.from_numpy_array(simiarity_matrix)
        scores=nx.pagerank(nx_graph,max_iter=600)
        top_sentence={sentence:scores[index] for index,sentence in enumerate(sentences)}
        sentNeeded= round(0.25*len(sentences))-1
        top=dict(sorted(top_sentence.items(), key=lambda x: x[1], reverse=True)[:sentNeeded])
        summary=""
        for sent in sentences:
          if sent in top.keys():
            summary+= sent
            return summary
  except:
    return float("NaN")


In [ ]:
import math
import csv
filename = "articlesSet.csv"
fields = ['title', 'summary', 'content']  # Correct variable name
with open(filename, 'a') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(fields)  # Use the correct variable name here

    def callback(row):
        summary = generateSummary(row['text'])
        if (type(summary) != str):
            return
        rows = [row['title'], summary, row['text']]
        csvwriter.writerow(rows)

    df.apply(callback, axis=1)


Summarising blog 1
Summarising blog 2
Summarising blog 3
Summarising blog 4
Summarising blog 5
Summarising blog 6
Summarising blog 7
Summarising blog 8
Summarising blog 9
Summarising blog 10
Summarising blog 11
Summarising blog 12
Summarising blog 13
Summarising blog 14
Summarising blog 15
Summarising blog 16
Summarising blog 17
Summarising blog 18
Summarising blog 19
Summarising blog 20
Summarising blog 21
Summarising blog 22
Summarising blog 23
Summarising blog 24
Summarising blog 25
Summarising blog 26
Summarising blog 27
Summarising blog 28
Summarising blog 29
Summarising blog 30
Summarising blog 31
Summarising blog 32
Summarising blog 33
Summarising blog 34
Summarising blog 35
Summarising blog 36
Summarising blog 37
Summarising blog 38
Summarising blog 39
Summarising blog 40
Summarising blog 41
Summarising blog 42
Summarising blog 43
Summarising blog 44
Summarising blog 45
Summarising blog 46
Summarising blog 47
Summarising blog 48
Summarising blog 49
Summarising blog 50
Summarisi